In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from collections import defaultdict
import pickle
from torch.autograd import Variable
import torch.optim as optim
import sys
import data_transform
import model_funcs
%matplotlib inline

In [ ]:
train_foldername = 'en-valid-10k'
train_filename = 'qa2_train'
train_fname = 'bAbI_Data/'+str(train_foldername)+'/'+str(train_filename)

valid_foldername = 'en-valid-10k'
valid_filename = 'qa2_valid'
valid_fname = 'bAbI_Data/'+str(valid_foldername)+'/'+str(valid_filename)

test_foldername = 'en-valid-10k'
test_filename = 'qa2_test'
test_fname = 'bAbI_Data/'+str(test_foldername)+'/'+str(test_filename)

unk_thres = 0
train_data_BOW, valid_data_BOW, test_data_BOW, train_data_pe, valid_data_pe, test_data_pe, vocab = data_transform.get_data(train_fname, valid_fname, test_fname, unk_thres = unk_thres)
print(train_data_BOW.shape)
print(valid_data_BOW.shape)
print(test_data_BOW.shape)
print(len(vocab))
#print(train_data_pe[0:5])

In [ ]:
embedding_dim = 30
vocab_size = len(vocab)
num_hops = 3
max_mem_size = 30
epochs = 20
eta = 0.005
LS = -1
ls_thres = 0.5
temporal = True
positional = True
same = 1

model = model_funcs.QuesAnsModel(embedding_dim, vocab_size, num_hops = num_hops, max_mem_size = max_mem_size,
                                 temporal=temporal, same=same, positional=positional)
l_tr, accuracy_tr, l_vd, accuracy_vd = model_funcs.train(model, train_data_BOW, valid_data_BOW,
                                                         train_data_pe, valid_data_pe,
                                                         epochs=epochs,eta=eta,opt=optim.Adam,LS=LS,ls_thres=ls_thres)

In [ ]:
acc = model_funcs.test(model,test_data_BOW,test_data_pe)